In [1]:
# %load ../uoicorr_imports.py
import h5py

import numpy as np
import matplotlib.pyplot as plt

import sys, os, pdb

# Hack to import pyuoi
parent_path, current_dir = os.path.split(os.path.abspath('.'))
while current_dir not in ['nse']:
    parent_path, current_dir = os.path.split(parent_path)
p = os.path.join(parent_path, current_dir)
# Add analysis
if p not in sys.path:
    sys.path.append(p)

import subprocess
hname = subprocess.check_output('hostname')

if 'ankitnse'.encode() in hname:

    if '%s/uoicorr' % p not in sys.path:
        sys.path.append('%s/uoicorr' % p)
    if '%s/PyUoI' % p not in sys.path:
        sys.path.append('%s/PyUoI' % p)

else:
    # And standard list of subdirectories
    if '%s\\pyuoi' % p not in sys.path:
        sys.path.append('%s\\pyuoi' % p)
    if '%s\\uoicorr' % p not in sys.path:
        sys.path.append('%s\\uoicorr' % p)

from postprocess import postprocess_file, postprocess_dir
from utils import FNR, FPR, selection_accuracy

from pyuoi.linear_model.gtv import GraphTotalVariance

In [2]:
gtv = GraphTotalVariance(np.random.uniform(), np.random.uniform(), np.random.uniform())

In [3]:
# Generate data to test the GTV. Use a block correlation structure and a small number of features
from utils import gen_covariance, gen_beta, gen_data

In [4]:
cov2 = gen_covariance('falloff', n_features = 30, L = 10)

In [5]:
np.linalg.matrix_rank(np.concatenate([cov2, np.identity(30)]))

30

In [6]:
cov = gen_covariance('block', n_features = 30, block_size = 5, correlation=0.5)
beta = gen_beta(n_features = 30, block_size = 5, sparsity = 0.5)
X, X_test, y, y_test = gen_data(n_samples = 100, n_features = 30, kappa = 0.3, covariance = cov, beta = beta)

In [8]:
gtv.fit(X, y, cov)

GraphTotalVariance(copy_X=True, fit_intercept=True,
          lambda_1=0.9137317719621729, lambda_S=0.4490300404300992,
          lambda_TV=0.5164258228179246, max_iter=1000, normalize=False,
          positive=False, precompute=False, random_state=None,
          selection='cyclic', tol=0.0001, warm_start=False)

In [9]:
from sklearn.linear_model import Lasso

In [10]:
l = Lasso(alpha = 0.9137)

In [11]:
l.fit(X, y)
l.coef_

array([ 8.792591  ,  0.48753341,  0.        , -0.96777973,  7.28692038,
        0.        ,  0.        ,  0.        ,  6.4006223 , -0.        ,
        0.9167634 ,  1.14467449,  0.        ,  0.43437247,  5.63113073,
        3.00554879,  5.93114585,  0.        ,  5.37543334,  0.        ,
        0.        , -1.4968608 ,  0.77093213,  9.88934712,  0.        ,
        1.9471279 ,  0.92732018,  4.59958914, 11.4708986 ,  0.        ])

In [12]:
gtv.coef_

array([4.02485823, 2.85519901, 2.70318306, 1.54537971, 3.52747765,
       0.51174788, 1.0197997 , 0.93373843, 2.08497666, 0.97112301,
       1.39121608, 1.8728707 , 0.99779037, 1.44196566, 2.77391377,
       2.9645662 , 3.87317908, 2.4102837 , 3.19678617, 2.19458331,
       1.63641299, 0.83340255, 1.80322228, 3.54989559, 1.63650855,
       4.20485001, 3.89419403, 3.94174546, 5.21155101, 3.30319634])